In [1]:
# mount drive
from google.colab import drive
drive.mount("/drive") 

Mounted at /drive


In [2]:
# imports
import numpy as np
import pandas as pd

In [3]:
# imports for plots
from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [4]:
# to make a detailed plot of decomposition
def plot_decompostion(decomposition_object, y, line_space):
  x = line_space
  d_object = decomposition_object

  fig = make_subplots(rows=3, cols=1,
                      subplot_titles=("Trend", "Seasonality", "Noise"))

  # time series & trend
  fig.add_trace(
      go.Scatter(x=x, y=y, name="initial time series"),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=x, y=d_object.trend, name="trend"),
      row=1, col=1
  )

  # seasonality
  fig.add_trace(
      go.Scatter(x=x, y=d_object.seasonal, name="seasonal"),
      row=2, col=1
  )

  # noise
  fig.add_trace(
      go.Scatter(x=x, y=d_object.resid, name="Noise"),
      row=3, col=1
  )


  fig.show()
  return

In [5]:
# to make plots of noise
def plot_noise(arr_noise):
  fig = make_subplots(rows=1, cols=2,
                      subplot_titles=("Histogram Noise", "Box plot of Noise"))
  
  fig.add_trace(
      go.Histogram(x=arr_noise),
      row=1, col=1
  )

  fig.add_trace(
      go.Box(x=arr_noise),
      row=1, col=2
  )
  
  fig.update_layout(showlegend=False)
  fig.show()
  return

In [6]:
# analyse
# decomposition of columns
# frequency: make the analysis for one year
# each year comprises 365 record
from statsmodels.tsa.seasonal import seasonal_decompose


def analyse_column(data_to_analyse, col_name, model="additive", freq=365):
  # decompose time series
  decomposition = seasonal_decompose(data_to_analyse[col_name], model=model, freq=freq, extrapolate_trend='freq')

  # decomposition plot
  plot_decompostion(decomposition, y=data_to_analyse[col_name], line_space=data_to_analyse["DATE"])

  # histogram and box plot of resid
  plot_noise(decomposition.resid)

  # mean and std of trend and noise
  dict_data = { "trend": decomposition.trend, "noise": decomposition.resid }
  for key, arr in dict_data.items():
    print(f"""
      =========================
      {key}
      mean: {np.mean(arr)}
      std: {np.std(arr)}
    """)

  return

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [7]:
# load data
data = pd.read_csv("/drive/My Drive/Colab Notebooks/AgriEdge/datasets/processed_weather.csv")

# convert DATE from str to timestamp
data["DATE"] = data["DATE"].apply(lambda s: pd.Timestamp(s))

In [8]:
data

,DATE,ALLSKY_SFC_SW_DWN,PRECTOT,RH2M,T2M,T2MDEW,T2M_MAX,T2M_MIN,WS2M,crop_year,day,T2M_AVG,GDD,cumulative_PREC
0,1981-10-15,-99.00,0.20,27.83,25.73,5.07,35.07,19.40,1.93,1982,1,27.235,27.235,0.20
1,1981-10-16,-99.00,0.14,46.26,23.85,11.38,30.86,17.82,2.78,1982,2,24.340,51.575,0.34
2,1981-10-17,-99.00,0.06,52.67,22.69,12.19,30.74,16.74,1.72,1982,3,23.740,75.315,0.40
3,1981-10-18,-99.00,0.28,37.36,25.51,9.55,32.46,21.01,3.36,1982,4,26.735,102.050,0.68
4,1981-10-19,-99.00,0.02,28.48,27.13,6.88,35.23,21.87,1.90,1982,5,28.550,130.600,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10486,2020-07-06,28.56,0.00,23.36,32.98,9.31,43.61,23.06,1.75,2020,265,33.335,4798.835,431.47
10487,2020-07-07,28.53,0.00,17.79,35.64,7.50,46.11,25.73,1.62,2020,266,35.920,4834.755,431.47
10488,2020-07-08,27.94,0.00,35.64,29.69,12.81,37.05,21.06,3.17,2020,267,29.055,4863.810,431.47
10489,2020-07-09,28.70,0.00,46.51,25.46,13.18,35.16,18.38,2.49,2020,268,26.770,4890.580,431.47


In [9]:
# select data to analyse

data_to_analyse = data[data["crop_year"] == 2005]

data_to_analyse

,DATE,ALLSKY_SFC_SW_DWN,PRECTOT,RH2M,T2M,T2MDEW,T2M_MAX,T2M_MIN,WS2M,crop_year,day,T2M_AVG,GDD,cumulative_PREC
6187,2004-10-15,11.02,0.02,67.59,17.49,11.38,24.14,13.54,1.92,2005,1,18.840,18.840,0.02
6188,2004-10-16,15.44,0.16,67.61,17.15,11.04,24.86,12.14,1.68,2005,2,18.500,37.340,0.18
6189,2004-10-17,18.47,0.00,55.29,18.32,9.08,27.04,11.63,1.55,2005,3,19.335,56.675,0.18
6190,2004-10-18,10.33,0.03,62.68,20.26,12.70,27.95,14.44,2.14,2005,4,21.195,77.870,0.21
6191,2004-10-19,10.55,0.11,55.43,23.38,13.85,31.29,18.06,2.24,2005,5,24.675,102.545,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6451,2005-07-06,25.56,0.00,39.26,28.14,12.59,38.10,18.72,1.87,2005,265,28.410,4675.575,288.22
6452,2005-07-07,28.58,0.00,42.66,28.70,14.57,38.18,20.69,2.38,2005,266,29.435,4705.010,288.22
6453,2005-07-08,23.90,0.03,48.11,26.94,14.83,36.35,19.94,2.50,2005,267,28.145,4733.155,288.25
6454,2005-07-09,26.68,0.03,48.61,27.55,15.67,36.64,19.82,2.47,2005,268,28.230,4761.385,288.28


In [10]:
fig = px.line(data_to_analyse, x="day", y="T2M_MAX")

fig.show()

In [11]:
# analyse over a period of 1 week
freq = 6

analyse_column(data_to_analyse, col_name="T2M_MAX", freq=freq)


      trend
      mean: 24.37156284298105
      std: 7.437647566950251
    

      noise
      mean: -0.02891402717652591
      std: 2.154650152092608
    
